In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch import tensor

from generate_data import generate_data, data_loader

# onehot = pd.Series(data = [tensor([1,0,0]),tensor([0,1,0]),tensor([0,0,1])], index = ['a','b','c'])


In [2]:
batch_size = 32

lr = 0.0001

loss_fn = nn.CrossEntropyLoss()


In [3]:
class LSTM_predictor(nn.Module):
    def __init__(self, SIZE):
        super(LSTM_predictor, self).__init__()

        self.SIZE = SIZE

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(3, SIZE)

        self.output = nn.Linear(SIZE, 2)

        self.activation = nn.Softmax()

        # self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, sentence):

        _, (lstm_out, _) = self.lstm(sentence)
        
        # print(lstm_out.shape)
        out = self.output(lstm_out[-1])
        # print(out.shape)
        # return self.softmax(out)


        # out = torch.sum(out,axis = 0)

        out = self.activation(out)

        return out
model = LSTM_predictor(32)

In [4]:
print(model)

LSTM_predictor(
  (lstm): LSTM(3, 32)
  (output): Linear(in_features=32, out_features=2, bias=True)
  (activation): Softmax(dim=None)
)


In [5]:
def test(model, X_test, y_test, min_length):
    model.eval()
    test_batches = data_loader(X_test,y_test,batch_size=batch_size, min_length = min_length)
    acc = 0.0
    i = 0
    for data, labels in test_batches:
        pred = model(data).view((-1,2))
        corr = (torch.argmax(pred,1) == torch.argmax(labels,1)).float().mean().item()
        acc += corr
        i += 1
    return acc/i

def train(model,n_epochs,lr, X, y, X_test, y_test, min_length):
    parameters = model.parameters()

    optimizer = torch.optim.Adam(parameters, lr=lr)

    model.train()

    losss, accs = [], []
    for i in range(n_epochs):
        model.train()
        train_batches = data_loader(X,y,batch_size=batch_size, min_length = min_length)

        sum_loss = 0.0
        i_loss = 0
        for data, labels in train_batches:
            pred = model(data)
            loss_fn = F.binary_cross_entropy
            loss = loss_fn(pred,labels)
            loss.backward()
            optimizer.step()

            sum_loss += loss.item()
            i_loss += 1

        losss.append(sum_loss/i_loss)
        accs.append(test(model,X_test, y_test, min_length))

        print(f"loss {sum_loss/i_loss:.3} - acc {test(model,X_test, y_test, min_length):.3}")

    return np.array(losss), np.array(accs)

# X_encoded[0]

In [6]:
N = 4*int(1e4)

min_length = 50

X_train, y_train, X_val, y_val = generate_data(0.8, is_short=False, type = 2)

X_test, y_test,_,_ = generate_data(0, is_short=False, type = 2)

TypeError: 'bool' object is not iterable

In [7]:
train(model, n_epochs = 10)


c:\Users\jakob\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


loss 0.616 - acc 0.647
loss 0.572 - acc 0.736
loss 0.533 - acc 0.685
loss 0.484 - acc 0.81
loss 0.406 - acc 0.806
loss 0.447 - acc 0.724
loss 0.495 - acc 0.827
loss 0.388 - acc 0.852
loss 0.335 - acc 0.881
loss 0.259 - acc 0.917


In [15]:
def train_and_test(model, lr):
    N = 4*int(1e4)

    min_length = 20

    X, y = generate_data(N, is_short=True, type = 2)

    X_test, y_test = generate_data(2000, is_short=True, type = 2)

    loss, acc = train(model, 50, lr, X, y, X_test, y_test, min_length = 20)
    
    min_length = 50

    X_test, y_test = generate_data(20000, is_short=False, type = 2)

    t = test(model, X_test, y_test, min_length = 50)
    final_acc = test
    print(f"Final acc {t:.3}")

    return loss, acc, final_acc


lrs = [0.0001, 0.0005, 0.001]
sizes = [16, 32]


save_i = 0
for lr in lrs:
    for size in sizes:
        losss, accs, faccs = [], [], []
        for i in range(3):
            model = LSTM_predictor(size)
            print("Training with lr", lr,"and size", size)
            loss, acc, facc = train_and_test(model, lr)

            data = {"lr":lr,'size':size,'run':i,'loss':loss,'acc':acc,'facc':facc}

            np.save(f"data{save_i}.npy",data, allow_pickle=True)
            save_i += 1



Training with lr 0.0001 and size 16


c:\Users\jakob\Documents\MLandData\ADL\Assignment 2\generate_data.py:116: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  alls = np.array([*true,*false])
c:\Users\jakob\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


loss 0.598 - acc 0.752
loss 0.485 - acc 0.624
loss 0.627 - acc 0.651
loss 0.509 - acc 0.713
loss 0.571 - acc 0.728
loss 0.517 - acc 0.791
loss 0.444 - acc 0.847
loss 0.374 - acc 0.844
loss 0.288 - acc 0.922
loss 0.243 - acc 0.927
loss 0.225 - acc 0.929
loss 0.195 - acc 0.957
loss 0.165 - acc 0.938
loss 0.207 - acc 0.941
loss 0.188 - acc 0.938
loss 0.177 - acc 0.918
loss 0.147 - acc 0.958
loss 0.134 - acc 0.97
loss 0.118 - acc 0.977
loss 0.189 - acc 0.946
Final acc 0.622
Training with lr 0.0001 and size 32
loss 0.594 - acc 0.828
loss 0.578 - acc 0.592
loss 0.518 - acc 0.788
loss 0.438 - acc 0.755
loss 0.461 - acc 0.84
loss 0.448 - acc 0.749
loss 0.475 - acc 0.789
loss 0.388 - acc 0.855
loss 0.319 - acc 0.882
loss 0.296 - acc 0.893
loss 0.225 - acc 0.94
loss 0.214 - acc 0.926
loss 0.193 - acc 0.939
loss 0.192 - acc 0.948
loss 0.223 - acc 0.941
loss 0.212 - acc 0.942
loss 0.185 - acc 0.953
loss 0.146 - acc 0.949
loss 0.148 - acc 0.955
loss 0.145 - acc 0.954
Final acc 0.681
Training with l

In [17]:
sum(range(21))

210